### Implement Black-Scholes pricing and greeks for European Call/Put Pricing to develop intuition for pricing/hedging formulas in complete markets

In [1]:
from scipy.stats import norm
import numpy as np
from typing import Mapping, Tuple

In [5]:
class EuroPricing:
    def __init__(self, typ: str, spot: float, strike: float, T: float, r: float, sigma: float):
        self.call = True if typ == 'call' else False
        self.spot = spot
        self.strike = strike
        self.T = T # expiration
        self.r = r # interest rate
        self.sigma = sigma
        self.price = self.get_price()
        self.greeks = self.get_greeks()

    def get_d1_d2(self):
        temp = self.sigma * np.sqrt(self.T)
        d1 = (np.log(self.spot/self.strike) + (self.r + self.sigma**2/2.)*self.T)/temp
        d2 = d1 - temp
        return d1, d2

    def get_price(self):
        d1, d2 = self.get_d1_d2()
        if self.call:
            return self.spot*norm.cdf(d1) - self.strike*np.exp(-self.r*self.T)*norm.cdf(d2)
        else:
            return self.strike*np.exp(-self.r*self.T)*norm.cdf(-d2) - self.spot*norm.cdf(-d1)

    def get_greeks(self):
        greeks = {}
        d1, d2 = self.get_d1_d2()
        temp = np.sqrt(self.T)

        greeks['gamma'] = norm.pdf(d1)/(self.spot*self.sigma*temp)
        vega = self.spot*temp*norm.pdf(d1)
        temp3 = -self.strike*self.T * np.exp(-self.r*self.T)
        temp1 = (self.spot*self.sigma*norm.pdf(d1))/(2*temp)
        temp2 = self.r*self.strike*np.exp(-self.r*self.T)

        if self.call:
            greeks['delta'] = norm.cdf(d1)
            greeks['theta'] = -temp1-temp2*norm.cdf(d2)
            greeks['rho'] = temp3*norm.cdf(d2)
        else:
            greeks['delta'] = -norm.cdf(-d1)
            greeks['theta'] = -temp1+temp2*norm.cdf(-d2)
            greeks['rho'] = temp3*norm.cdf(-d2)

        return greeks

In [6]:
example = EuroPricing(typ="call", spot=100.0, strike=120.0, T=3.0, r=0.05, sigma=0.2)
print("The option price is ", example.price)
print("The greeks are ", example.greeks)

The option price is  12.39453808999707
The greeks are  {'gamma': 0.011479768903287241, 'delta': 0.5318419165318319, 'theta': -4.335436458816755, 'rho': -122.36896068955838}
